<a href="https://colab.research.google.com/github/WonJunPark/BiSeNet_colab_inference/blob/master/BiSeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Sep 29 18:13:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
print(torch.__version__)

1.6.0+cu101


In [3]:
%cd /content
!git clone https://github.com/CoinCheung/BiSeNet.git

/content
Cloning into 'BiSeNet'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 238 (delta 28), reused 69 (delta 21), pack-reused 156
Receiving objects: 100% (238/238), 2.42 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (96/96), done.


# test

In [4]:
%cd /content/BiSeNet/
!dir

/content/BiSeNet
configs  datasets  example.png	lib  old  README.md  tensorrt  tools


In [5]:
!python ./tools/demo.py --model bisenetv2 --weight-path ./model_final_v2.pth --img-path ./example.png

In [10]:
%cd /content/BiSeNet/weight/
!dir

/content/BiSeNet/weight
model_final_v2.pth


# inference

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [35]:
%cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [36]:
import os

img_object = 'cars'
img_path = './semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

In [37]:
print(img_list)

['18132.png', '18121.png', '18227.png', '18212.png', '18156.png', '18133.png', '18135.png', '18235.png', '18158.png', '18052.png', '18165.png', '18065.png', '18106.png', '18170.png', '18148.png', '18085.png', '18245.png', '18092.png', '18228.png', '18151.png', '18113.png', '18066.png', '18248.png', '18083.png', '18268.png', '18215.png', '18175.png', '18186.png', '18098.png', '18099.png', '18196.png', '18258.png', '18110.png', '18126.png', '18198.png', '18253.png', '18108.png', '18266.png', '18246.png', '18104.png', '18252.png', '18240.png', '18077.png', '18075.png', '18050.png', '18137.png', '18081.png', '18194.png', '18090.png', '18087.png', '18091.png', '18213.png', '18237.png', '18129.png', '18209.png', '18149.png', '18219.png', '18160.png', '18242.png', '18057.png', '18143.png', '18088.png', '18161.png', '18249.png', '18251.png', '18250.png', '18171.png', '18069.png', '18155.png', '18169.png', '18072.png', '18254.png', '18114.png', '18105.png', '18140.png', '18220.png', '18063.png'

In [38]:
%cd /content/BiSeNet
!dir

/content/BiSeNet
configs   example.png  model_final_v2.pth  README.md  result	tools
datasets  lib	       old		   res.jpg    tensorrt


In [48]:
!cd ../gdrive/My\ Drive/semantic_test

## cars

In [52]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'cars'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.032866716384887695
0.03586101531982422
0.03723883628845215
0.03384113311767578
0.03461909294128418
0.03611159324645996
0.0335383415222168
0.03789234161376953
0.03416895866394043
0.033553123474121094
0.0356290340423584
0.03695392608642578
0.03254270553588867
0.03287506103515625
0.05495142936706543
0.03369569778442383
0.03741621971130371
0.03657388687133789
0.03354144096374512
0.03812432289123535
0.036054134368896484
0.03408670425415039
0.03490257263183594
0.039327144622802734
0.035466670989990234
0.04264116287231445
0.033762216567993164
0.03568387031555176
0.03469586372375488
0.039199113845825195
0.0337522029876709
0.04930305480957031
0.035594940185546875
0.033743858337402344
0.03645038604736328
0.037775278091430664
0.034090280532836914
0.046381235122680664
0.03424358367919922
0.03428220748901367
0.034340620040893555
0.03579306602478027
0.03537344932556152
0.03615927696228027
0.035044193267822266
0.03443121910095215
0.03518056869506836
0.034778594970703125
0.03547787666320801
0.037892

In [53]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'crowded'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.03397250175476074
0.03639721870422363
0.03588604927062988
0.03449440002441406
0.03480696678161621
0.036173105239868164
0.03413963317871094
0.03334927558898926
0.03544187545776367
0.03433036804199219
0.03552818298339844
0.040689706802368164
0.035486459732055664
0.03793048858642578
0.036947011947631836
0.037233829498291016
0.03544449806213379
0.04102301597595215
0.0351109504699707
0.03652310371398926
0.03639674186706543
0.035249948501586914
0.03386974334716797
0.03569221496582031
0.03434300422668457
0.03583574295043945
0.037760257720947266
0.03771662712097168
0.03691554069519043
0.03504776954650879
0.03538775444030762
0.04675912857055664
0.036242008209228516
0.03366708755493164
0.03599429130554199
0.036115169525146484
0.038001298904418945
0.037194252014160156
0.03712153434753418
0.035308122634887695
0.036216020584106445
0.036397457122802734
0.03411507606506348
0.03642153739929199
0.03273510932922363
0.03456306457519531
0.03592276573181152
0.03626537322998047
0.04138040542602539
0.03304

## pedestrians

In [54]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'pedestrians'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.03483152389526367
0.03462791442871094
0.03598928451538086
0.03978276252746582
0.0354771614074707
0.034644126892089844
0.03649306297302246
0.03856348991394043
0.03597879409790039
0.03567314147949219
0.034549713134765625
0.03394508361816406
0.03540372848510742
0.035674333572387695
0.034485578536987305
0.034174442291259766
0.03785276412963867
0.03792691230773926
0.038658857345581055
0.03336668014526367
0.034226417541503906
0.035010337829589844
0.034844398498535156
0.03728175163269043
0.04168510437011719
test time:  40.06355905532837
